**Table of contents**<a id='toc0_'></a>    
- [Step 1: Building a basic E-Commerce chatbot with Langchain](#toc1_)    
- [Step 2: Enhancing Conversations with Memory using ConversationBufferMemory](#toc2_)    
- [Task 3: ConversationSummaryBufferMemory Implementation with Azure OpenAI](#toc3_)    
- [Task 4: Multi-Step Workflows using SequentialChain with Azure OpenAI](#toc4_)    
- [Task 5: Combining Memory and Chains for a context aware multi step chatbot](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain import __version__ as langchain_version
print(f"LangChain version: {langchain_version}")

# <a id='toc1_'></a>[Step 1: Building a basic E-Commerce chatbot with Langchain](#toc0_)

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
# Azure OpenAI setup (update deployment name if needed)
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.3
)

In [ ]:
order_tracking_prompt = ChatPromptTemplate.from_template(
    "A customer wants to check the status of order ID {order_id}. Provide a response."
)

In [ ]:
order_tracking_chain = order_tracking_prompt | llm

In [ ]:
response = order_tracking_chain.invoke({"order_id": "ORD12345"})
print(response.content)

In [ ]:
product_inquiry_prompt = ChatPromptTemplate.from_template(
    "Provide details about the product: {product_name}."
)

refund_policy_prompt = ChatPromptTemplate.from_template(
    "A customer asked: {issue}. Provide information about the refund policy."
)

In [ ]:
# Deprecated:
# product_inquiry_chain = LLMChain(llm=llm, prompt=product_inquiry_prompt)
# refund_policy_chain = LLMChain(llm=llm, prompt=refund_policy_prompt)
product_inquiry_chain = product_inquiry_prompt | llm
refund_policy_chain = refund_policy_prompt | llm

In [ ]:
response = product_inquiry_chain.invoke({"product_name": "Wireless Headphones"})
print(response.content)

In [ ]:
response = refund_policy_chain.invoke({"issue": "How can I return a damaged item?"})
print(response.content)

In [ ]:
# Import required libraries for conversation memory and chain
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferMemoryfrom langchain.chains import ConversationChainfrom langchain.prompts import PromptTemplate

# <a id='toc2_'></a>[Step 2: Enhancing Conversations with Memory using ConversationBufferMemory](#toc0_)

In [60]:
import os
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

In [61]:
# Initialize ConversationBufferMemory
memory = ConversationBufferMemory()

In [62]:
# Initialize the Azure Language Model (LLM)
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.3
)

In [63]:
# Create a custom prompt template for customer service role
template = """You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- General help: Be supportive and offer solutions

Current conversation:
{history}
Human: {input}
Assistant:"""

PROMPT = PromptTemplate(
    input_variables=["history", "input"],
    template=template
)

In [64]:
# Create a ConversationChain with Memory and Custom Prompt
conversation_chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=PROMPT,
    verbose=True
)

In [65]:
# Start a conversation and store history
print(conversation_chain.invoke("Hi, I placed an order for a smartphone. The order id is #12345. Can you check the status?")['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- General help: Be supportive and offer solutions

Current conversation:

Human: Hi, I placed an order for a smartphone. The order id is #12345. Can you check the status?
Assistant:

> Finished chain.
Sure! Let me check the status of your order #12345 for the smartphone. One moment, please.

[Checking order status...]

Your order is currently being processed and is scheduled to ship out within the next 24 hours. You will receive a confirmat

In [66]:
# Continue conversation with context
print(conversation_chain.invoke("Can you expedite the delivery?")['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- General help: Be supportive and offer solutions

Current conversation:
Human: Hi, I placed an order for a smartphone. The order id is #12345. Can you check the status?
AI: Sure! Let me check the status of your order #12345 for the smartphone. One moment, please.

[Checking order status...]

Your order is currently being processed and is scheduled to ship out within the next 24 hours. You will receive a confirmation email with tracking inf

In [67]:
# Inspect the stored memory
print("Stored Memory:", memory.buffer)

Stored Memory: Human: Hi, I placed an order for a smartphone. The order id is #12345. Can you check the status?
AI: Sure! Let me check the status of your order #12345 for the smartphone. One moment, please.

[Checking order status...]

Your order is currently being processed and is scheduled to ship out within the next 24 hours. You will receive a confirmation email with tracking information once it has shipped. If you have any other questions or need further assistance, feel free to ask!
Human: Can you expedite the delivery?
AI: I can certainly help with that! Let me check the options available for expediting your delivery. One moment, please.

[Checking delivery options...]

It looks like we can expedite your delivery to ensure it arrives sooner. There may be an additional shipping fee for this service. Would you like me to proceed with expediting your order? If so, please confirm, and I'll take care of it for you!


In [68]:
# Test contextual understanding
print(conversation_chain.invoke("Remind me what product I ordered?")['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- General help: Be supportive and offer solutions

Current conversation:
Human: Hi, I placed an order for a smartphone. The order id is #12345. Can you check the status?
AI: Sure! Let me check the status of your order #12345 for the smartphone. One moment, please.

[Checking order status...]

Your order is currently being processed and is scheduled to ship out within the next 24 hours. You will receive a confirmation email with tracking inf

In [69]:
# Clear memory and reset conversation
memory.clear()
print("Memory cleared.")

Memory cleared.


In [70]:
# Test behavior after memory is cleared
print(conversation_chain.invoke("Can you check my order again?")['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- General help: Be supportive and offer solutions

Current conversation:

Human: Can you check my order again?
Assistant:

> Finished chain.
Of course! Could you please provide me with your order number or the email address associated with your account? This will help me locate your order and provide you with the latest updates.

> Finished chain.
Of course! Could you please provide me with your order number or the email address associated 

In [71]:
# Inspect the stored memory
print("Stored Memory:", memory.buffer)

Stored Memory: Human: Can you check my order again?
AI: Of course! Could you please provide me with your order number or the email address associated with your account? This will help me locate your order and provide you with the latest updates.


# <a id='toc3_'></a>[Task 3: ConversationSummaryBufferMemory Implementation with Azure OpenAI](#toc0_)

In [74]:
# Import required modules for summary memory
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
load_dotenv()

True

In [75]:
# Initialize the Azure Language Model (LLM)
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.3
)

In [76]:
# Initialize ConversationBufferWindowMemory (keeps last k exchanges)
summary_memory = ConversationBufferWindowMemory(k=5)

In [77]:
# Create a custom prompt template for customer service role
template = """You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- Address changes: You can update delivery addresses
- Returns/Refunds: You can process returns and explain timelines
- Product recommendations: You can suggest relevant products

Current conversation:
{history}
Human: {input}
Assistant:"""

PROMPT = PromptTemplate(
    input_variables=["history", "input"],
    template=template
)

In [78]:
# Create a ConversationChain with summary memory
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=summary_memory,
    prompt=PROMPT,
    verbose=True
)

In [79]:
# Start a detailed customer support conversation
print(conversation_with_summary.invoke({
    "input": "Hi, I placed an order for a 55-inch 4K OLED TV last week with order ID ORD98234. The expected delivery was supposed to be yesterday, but I haven't received any updates. Can you check the status for me?"
})['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- Address changes: You can update delivery addresses
- Returns/Refunds: You can process returns and explain timelines
- Product recommendations: You can suggest relevant products

Current conversation:

Human: Hi, I placed an order for a 55-inch 4K OLED TV last week with order ID ORD98234. The expected delivery was supposed to be yesterday, but I haven't received any updates. Can you check the status for me?
Assistant:

> Finished chain.
Su

In [80]:
print(conversation_with_summary.invoke({
    "input": "Also, I realized that I might have provided the wrong delivery address. Can you confirm the address associated with my order? If it's incorrect, I'd like to update it to my new address at 123 Elm Street, Springfield."
})['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- Address changes: You can update delivery addresses
- Returns/Refunds: You can process returns and explain timelines
- Product recommendations: You can suggest relevant products

Current conversation:
Human: Hi, I placed an order for a 55-inch 4K OLED TV last week with order ID ORD98234. The expected delivery was supposed to be yesterday, but I haven't received any updates. Can you check the status for me?
AI: Sure! Let me check the status

In [81]:
print(conversation_with_summary.invoke({
    "input": "While you're at it, I was also looking at the 65-inch variant of the same TV. Can you tell me if there are any discounts or bundle offers available for that model?"
})['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- Address changes: You can update delivery addresses
- Returns/Refunds: You can process returns and explain timelines
- Product recommendations: You can suggest relevant products

Current conversation:
Human: Hi, I placed an order for a 55-inch 4K OLED TV last week with order ID ORD98234. The expected delivery was supposed to be yesterday, but I haven't received any updates. Can you check the status for me?
AI: Sure! Let me check the status

In [82]:
print(conversation_with_summary.invoke({
    "input": "If I decide to cancel the 55-inch TV and order the 65-inch one instead, what would be the process, and how long would the refund take? Also, will I be charged any cancellation fees?"
})['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- Address changes: You can update delivery addresses
- Returns/Refunds: You can process returns and explain timelines
- Product recommendations: You can suggest relevant products

Current conversation:
Human: Hi, I placed an order for a 55-inch 4K OLED TV last week with order ID ORD98234. The expected delivery was supposed to be yesterday, but I haven't received any updates. Can you check the status for me?
AI: Sure! Let me check the status

In [83]:
# Inspect the generated conversation summary
print("\nConversation Summary:", summary_memory.buffer)


Conversation Summary: Human: Hi, I placed an order for a 55-inch 4K OLED TV last week with order ID ORD98234. The expected delivery was supposed to be yesterday, but I haven't received any updates. Can you check the status for me?
AI: Sure! Let me check the status of your order for the 55-inch 4K OLED TV with order ID ORD98234. One moment, please.

[After checking the system]

It looks like your order was delayed in transit, but it is currently on its way to you. The latest update shows that it is scheduled to be delivered by the end of the day tomorrow. I apologize for the inconvenience and appreciate your patience. If you need any further assistance or have any questions, feel free to ask!
Human: Also, I realized that I might have provided the wrong delivery address. Can you confirm the address associated with my order? If it's incorrect, I'd like to update it to my new address at 123 Elm Street, Springfield.
AI: I can help with that! Let me check the address associated with your or

In [84]:
# Test follow-up queries using summarized memory
print(conversation_with_summary.invoke({
    "input": "Can you confirm if my address has been updated successfully?"
})['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- Address changes: You can update delivery addresses
- Returns/Refunds: You can process returns and explain timelines
- Product recommendations: You can suggest relevant products

Current conversation:
Human: Hi, I placed an order for a 55-inch 4K OLED TV last week with order ID ORD98234. The expected delivery was supposed to be yesterday, but I haven't received any updates. Can you check the status for me?
AI: Sure! Let me check the status

In [85]:
print(conversation_with_summary.invoke({
    "input": "What did you say about the refund timeline again?"
})['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- Address changes: You can update delivery addresses
- Returns/Refunds: You can process returns and explain timelines
- Product recommendations: You can suggest relevant products

Current conversation:
Human: Hi, I placed an order for a 55-inch 4K OLED TV last week with order ID ORD98234. The expected delivery was supposed to be yesterday, but I haven't received any updates. Can you check the status for me?
AI: Sure! Let me check the status

In [86]:
# Simulate exceeding the token limit
for i in range(5):
    print(conversation_with_summary.invoke({
        "input": f"By the way, I was also considering adding a soundbar to my order. What options do you recommend for premium models with Dolby Atmos? (Query {i+1})"
    })['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- Address changes: You can update delivery addresses
- Returns/Refunds: You can process returns and explain timelines
- Product recommendations: You can suggest relevant products

Current conversation:
Human: Also, I realized that I might have provided the wrong delivery address. Can you confirm the address associated with my order? If it's incorrect, I'd like to update it to my new address at 123 Elm Street, Springfield.
AI: I can help wit

In [87]:
# Review how the summary adapts
print("\nUpdated Conversation Summary:", summary_memory.buffer)


Updated Conversation Summary: Human: By the way, I was also considering adding a soundbar to my order. What options do you recommend for premium models with Dolby Atmos? (Query 1)
AI: Great choice! For premium soundbars with Dolby Atmos, I recommend the following options:

1. **Sonos Arc**: This soundbar offers exceptional sound quality with Dolby Atmos support. It features a sleek design, built-in voice control, and integrates seamlessly with other Sonos products for a multi-room audio experience.

2. **Samsung HW-Q950A**: This model provides a powerful surround sound experience with 11.1.4 channels and supports Dolby Atmos. It comes with a wireless subwoofer and rear speakers for an immersive audio experience.

3. **LG SN11RG**: This soundbar includes Dolby Atmos and DTS:X support, along with a wireless subwoofer and rear speakers for a full surround sound setup. It also features AI Sound Pro technology for optimized audio performance.

4. **Bose Smart Soundbar 900**: This soundbar 

In [88]:
# Clear memory and start fresh
summary_memory.clear()
print("Memory cleared. Starting fresh conversation.\n")

Memory cleared. Starting fresh conversation.



In [89]:
print(conversation_with_summary.invoke({
    "input": "I received my order, but the TV box was damaged. I need help with initiating a return and getting a replacement."
})['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have access to order systems and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide updates
- Delivery: You can provide tracking info and expedite if needed
- Products: You remember what they ordered and can provide details
- Address changes: You can update delivery addresses
- Returns/Refunds: You can process returns and explain timelines
- Product recommendations: You can suggest relevant products

Current conversation:

Human: I received my order, but the TV box was damaged. I need help with initiating a return and getting a replacement.
Assistant:

> Finished chain.
I'm sorry to hear that your TV box arrived damaged. I can definitely help you with initiati

In [90]:
# Review how the summary adapts
print("\nUpdated Conversation Summary:", summary_memory.buffer)


Updated Conversation Summary: Human: I received my order, but the TV box was damaged. I need help with initiating a return and getting a replacement.
AI: I'm sorry to hear that your TV box arrived damaged. I can definitely help you with initiating a return and arranging for a replacement. 

To get started, could you please provide me with your order number? Once I have that, I can process the return and set up the replacement for you. Thank you!


# <a id='toc4_'></a>[Task 4: Multi-Step Workflows using SequentialChain with Azure OpenAI](#toc0_)

In [94]:
# Initialize the Azure Language Model (LLM)
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.3
)

In [95]:
# Create prompt template and LLM chain for order verification
order_verification_prompt = ChatPromptTemplate.from_template(
    "You are a customer service representative with access to order systems. "
    "A customer provided order ID {order_id}. Look up and provide specific product details "
    "including product name, price, order date, and delivery date. "
    "Create realistic order details for this order ID."
)

order_verification_chain = LLMChain(llm=llm, prompt=order_verification_prompt, output_key="order_details")

In [96]:
# Create prompt template and LLM chain for return initiation
return_initiation_prompt = ChatPromptTemplate.from_template(
    "You are a customer service representative helping with returns. "
    "Based on the following order details: {order_details}, "
    "explain the return eligibility, process steps, and requirements specific to this order. "
    "Be helpful and provide clear instructions."
)

return_initiation_chain = LLMChain(llm=llm, prompt=return_initiation_prompt, output_key="return_process")

In [97]:
# Create prompt template and LLM chain for refund timeline
refund_processing_prompt = PromptTemplate(
    input_variables=["return_process"],
    template="You are a customer service representative providing refund information. "
    "Based on the return process: {return_process}, "
    "provide the specific estimated refund timeline and processing details for this customer."
)

refund_processing_chain = LLMChain(llm=llm, prompt=refund_processing_prompt, output_key="refund_details")

In [98]:
# Combine all chains into a sequential workflow
multi_step_workflow = SequentialChain(
    chains=[order_verification_chain, return_initiation_chain, refund_processing_chain],
    input_variables=["order_id"],
    output_variables=["order_details", "return_process", "refund_details"],
    verbose=True
)

In [99]:
# Test the workflow with a sample order
order_id = "ORD67890"
response = multi_step_workflow.invoke({"order_id": order_id})

print("\nOrder Details:", response["order_details"])
print("\nReturn Process:", response["return_process"])
print("\nRefund Details:", response["refund_details"])



> Entering new SequentialChain chain...

> Finished chain.

Order Details: Here are the details for order ID ORD67890:

- **Product Name:** Wireless Noise-Canceling Headphones
- **Price:** $129.99
- **Order Date:** September 15, 2023
- **Delivery Date:** September 22, 2023

If you need any further assistance or information, feel free to ask!

Return Process: Thank you for reaching out regarding your order ID ORD67890 for the Wireless Noise-Canceling Headphones. I’m here to assist you with the return process. Here’s what you need to know:

### Return Eligibility
1. **Return Window**: You can return the headphones within 30 days of the delivery date. Since your delivery date was September 22, 2023, you have until October 22, 2023, to initiate a return.
2. **Condition of the Product**: The headphones must be in their original condition, unused, and in the original packaging. All accessories, manuals, and any promotional items must also be included.
3. **Non-Returnable Items**: If the pr

In [100]:
# Wrap the workflow into a reusable function with input validation
def process_order_workflow(order_id):
    if order_id == '' or order_id == None:
        return {'error': 'Kindly provide your order id'}
    return multi_step_workflow.invoke({"order_id": order_id})

# Example usage
print("\n" + "="*60)
print("TESTING REUSABLE FUNCTION")
print("="*60)
print(process_order_workflow("ORD54321"))

print("\n" + "="*60)
print("TESTING ERROR HANDLING")
print("="*60)
print(process_order_workflow(""))
print(process_order_workflow(None))


TESTING REUSABLE FUNCTION


> Entering new SequentialChain chain...

> Finished chain.
{'order_id': 'ORD54321', 'order_details': 'Sure! Here are the details for order ID ORD54321:\n\n- **Product Name:** Wireless Bluetooth Headphones\n- **Price:** $89.99\n- **Order Date:** September 15, 2023\n- **Delivery Date:** September 22, 2023\n\nIf you have any further questions or need assistance with anything else, feel free to ask!', 'return_process': 'Certainly! Here’s the information regarding the return eligibility, process steps, and requirements for your order of the Wireless Bluetooth Headphones (Order ID: ORD54321).\n\n### Return Eligibility:\n1. **Return Window:** You can return the product within 30 days of the delivery date. Since your headphones were delivered on September 22, 2023, you have until October 22, 2023, to initiate a return.\n2. **Condition of the Product:** The headphones must be in their original condition, unused, and in the original packaging with all accessories inc

# <a id='toc5_'></a>[Task 5: Combining Memory and Chains for a context aware multi step chatbot](#toc0_)

In [104]:
# Import required modules for context-aware multi-intent chatbot
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import SequentialChain, LLMChain, ConversationChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from dotenv import load_dotenv
import os
load_dotenv()

True

In [105]:
# Initialize the Azure Language Model (LLM)
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.3
)

In [106]:
# Initialize ConversationBufferWindowMemory
summary_memory = ConversationBufferWindowMemory(k=5)

In [107]:
# Create prompt templates for multiple intents
order_tracking_prompt = ChatPromptTemplate.from_template(
    "You are a customer service representative with access to order systems. "
    "Check the status of order ID {order_id}. Provide realistic order status, "
    "expected delivery date and any delay updates for this specific order."
)

product_inquiry_prompt = ChatPromptTemplate.from_template(
    "You are a customer service representative with access to product databases. "
    "Provide details about the product {product_name}, including specifications, "
    "pricing, and current stock availability. Be helpful and specific."
)

refund_policy_prompt = ChatPromptTemplate.from_template(
    "You are a customer service representative helping with returns and refunds. "
    "A customer wants to return an item. Based on their request: {return_request}, "
    "explain the return process and refund policy clearly and helpfully."
)

In [108]:
# Create LLMChains for each intent
order_tracking_chain = LLMChain(llm=llm, prompt=order_tracking_prompt, output_key="order_status")
product_inquiry_chain = LLMChain(llm=llm, prompt=product_inquiry_prompt, output_key="product_info")
refund_policy_chain = LLMChain(llm=llm, prompt=refund_policy_prompt, output_key="refund_details")

In [109]:
# Build a combined workflow with memory
combined_workflow = SequentialChain(
    chains=[order_tracking_chain, product_inquiry_chain, refund_policy_chain],
    input_variables=["order_id", "product_name", "return_request"],
    output_variables=["order_status", "product_info", "refund_details"],
    verbose=True
)

In [110]:
# Create a ConversationChain with memory and custom prompt
conversation_template = """You are a helpful customer service representative for an online electronics store. You have full access to order systems, product databases, and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide realistic updates with specific details
- Products: You have access to product information and can provide specifications and pricing
- Returns/Refunds: You can process returns and explain timelines
- General help: Be supportive and offer solutions

Current conversation:
{history}
Human: {input}
Assistant:"""

CONVERSATION_PROMPT = PromptTemplate(
    input_variables=["history", "input"], 
    template=conversation_template
)

conversation_chain = ConversationChain(
    llm=llm, 
    memory=summary_memory, 
    prompt=CONVERSATION_PROMPT,
    verbose=True
)

In [111]:
# Start a conversation with memory tracking
print("=== CONVERSATION 1: Order Status Check ===")
print(conversation_chain.invoke({
    "input": "Hi, I ordered a PlayStation 5 with order ID ORD67890. It was supposed to arrive yesterday but hasn't. Can you check the status?"
})['response'])

=== CONVERSATION 1: Order Status Check ===


> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have full access to order systems, product databases, and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide realistic updates with specific details
- Products: You have access to product information and can provide specifications and pricing
- Returns/Refunds: You can process returns and explain timelines
- General help: Be supportive and offer solutions

Current conversation:

Human: Hi, I ordered a PlayStation 5 with order ID ORD67890. It was supposed to arrive yesterday but hasn't. Can you check the status?
Assistant:

> Finished chain.
Sure! Let me check the status of your order for the PlayStation 5 with order I

In [112]:
print("\n=== CONVERSATION 2: Product Inquiry ===")
print(conversation_chain.invoke({
    "input": "Also, I'm interested in buying a gaming headset. What options do you have?"
})['response'])


=== CONVERSATION 2: Product Inquiry ===


> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have full access to order systems, product databases, and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide realistic updates with specific details
- Products: You have access to product information and can provide specifications and pricing
- Returns/Refunds: You can process returns and explain timelines
- General help: Be supportive and offer solutions

Current conversation:
Human: Hi, I ordered a PlayStation 5 with order ID ORD67890. It was supposed to arrive yesterday but hasn't. Can you check the status?
AI: Sure! Let me check the status of your order for the PlayStation 5 with order ID ORD67890. 

It looks like y

In [113]:
print("\n=== CONVERSATION 3: Return Inquiry ===")
print(conversation_chain.invoke({
    "input": "If I change my mind about the PlayStation 5, how can I return it, and how long does the refund take?"
})['response'])


=== CONVERSATION 3: Return Inquiry ===


> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have full access to order systems, product databases, and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide realistic updates with specific details
- Products: You have access to product information and can provide specifications and pricing
- Returns/Refunds: You can process returns and explain timelines
- General help: Be supportive and offer solutions

Current conversation:
Human: Hi, I ordered a PlayStation 5 with order ID ORD67890. It was supposed to arrive yesterday but hasn't. Can you check the status?
AI: Sure! Let me check the status of your order for the PlayStation 5 with order ID ORD67890. 

It looks like yo

In [115]:
print("\n=== CONVERSATION 4: History Question ===")
print(conversation_chain.invoke({
    "input": "What is my playsatation 5 order id?"
})['response'])


=== CONVERSATION 4: History Question ===


> Entering new ConversationChain chain...
Prompt after formatting:
You are a helpful customer service representative for an online electronics store. You have full access to order systems, product databases, and can help customers with their inquiries. You should provide helpful, specific responses about orders, deliveries, and products. Be friendly and professional.

When customers ask about:
- Order status: You can check and provide realistic updates with specific details
- Products: You have access to product information and can provide specifications and pricing
- Returns/Refunds: You can process returns and explain timelines
- General help: Be supportive and offer solutions

Current conversation:
Human: Hi, I ordered a PlayStation 5 with order ID ORD67890. It was supposed to arrive yesterday but hasn't. Can you check the status?
AI: Sure! Let me check the status of your order for the PlayStation 5 with order ID ORD67890. 

It looks like 

In [116]:
# Check the conversation summary
print("\n" + "="*60)
print("CONVERSATION MEMORY CHECK")
print("="*60)
print("Conversation Summary:", summary_memory.buffer)


CONVERSATION MEMORY CHECK
Conversation Summary: Human: Hi, I ordered a PlayStation 5 with order ID ORD67890. It was supposed to arrive yesterday but hasn't. Can you check the status?
AI: Sure! Let me check the status of your order for the PlayStation 5 with order ID ORD67890. 

It looks like your order was shipped on the expected date, but there seems to be a delay with the carrier. The latest update indicates that it is currently in transit and should arrive within the next 1-2 business days. I apologize for the inconvenience and appreciate your patience. 

If you have any further questions or need assistance with anything else, feel free to ask!
Human: Also, I'm interested in buying a gaming headset. What options do you have?
AI: We have a great selection of gaming headsets! Here are a few popular options:

1. **HyperX Cloud II Gaming Headset**
   - **Price:** $99.99
   - **Features:** 53mm drivers for immersive sound, detachable microphone, and memory foam ear cushions for comfort 

In [117]:
# Run the workflow with integrated memory-based context
print("\n" + "="*60)
print("RUNNING CONTEXT-AWARE WORKFLOW")
print("="*60)

# Extract context from conversation memory to make workflow context-aware
conversation_context = summary_memory.buffer if summary_memory.buffer else ""

# Create context-aware inputs that reference the conversation
context_aware_return_request = f"Customer from previous conversation (who ordered PlayStation 5 with order ORD67890) wants to return the PlayStation 5 because they changed their mind."

response = combined_workflow.invoke({
    "order_id": "ORD67890",  # From the conversation
    "product_name": "Wireless Gaming Headset",  # New inquiry
    "return_request": context_aware_return_request  # Context-aware
})

print("\nOrder Tracking Response:", response["order_status"])
print("\nProduct Inquiry Response:", response["product_info"])
print("\nRefund Policy Response:", response["refund_details"])


RUNNING CONTEXT-AWARE WORKFLOW


> Entering new SequentialChain chain...

> Finished chain.

Order Tracking Response: I'm sorry, but I don't have access to order systems or the ability to check specific order statuses. However, I can guide you on how to check your order status. You can typically do this by visiting the website of the retailer where you placed the order, logging into your account, and navigating to the order history section. Alternatively, you can contact their customer service directly for assistance. If you have any other questions or need further help, feel free to ask!

Product Inquiry Response: I’m sorry, but I don’t have access to real-time product databases or current stock availability. However, I can provide you with a general overview of what you might expect from a typical wireless gaming headset, including common specifications and features. 

### Wireless Gaming Headset Overview

**Specifications:**
- **Audio Drivers:** Usually 40mm to 50mm neodymium drive